# Reflectionのフォーマット

In [88]:
BASE_PATH = '/home/masa1357/Dockerdata/gitfile/kaggle_clrp_1st_place_solution'
DATA_PATH = BASE_PATH + '/Reflection_Marged'

OUT_PATH = BASE_PATH + '/Reflection_Embedded'

In [89]:
import pandas as pd
import os
import logging

In [90]:
logger = logging.getLogger("my_logger")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
logger.addHandler(handler)

In [91]:
file = [f for f in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, f))]
file

['Course_D-2020_Marged.csv',
 'Course_C-2022-1_Marged.csv',
 'Course_B-2020_Marged.csv',
 'Course_C-2021-1_Marged.csv',
 'Course_C-2021-2_Marged.csv',
 'Course_D-2022_Marged.csv',
 'Course_D-2021_Marged.csv']

In [92]:
df = pd.read_csv(os.path.join(DATA_PATH, file[0]))
df

,userid,course_number,question_number,question_content,answer_content,grade
0,D-2020_U51,1,1,今日の内容を自分なりの言葉で説明してみてください,黄色のｍoodleにログインして、そのあと水色のmoodleにリンク、授業開始までにTeam...,F
1,D-2020_U51,1,2,今日の内容で、分かったこと・できたことを書いてください,授業の進め方が理解できた。,F
2,D-2020_U51,1,3,今日の内容で、分からなかったこと・できなかったことを書いてください,とくになし,F
3,D-2020_U51,1,4,質問があれば書いてください,とくになし,F
4,D-2020_U51,1,5,今日の授業の感想や反省を書いてください,オンライン授業ならではの利点を活用する工夫で、最初は大丈夫かと不安だったが何とかなりそうだと...,F
...,...,...,...,...,...,...
2600,D-2020_U43,11,1,今日の内容を自分なりの言葉で説明してみてください,デジタルフィルタについて,A
2601,D-2020_U43,11,2,今日の内容で、分かったこと・できたことを書いてください,フィルタの種類と設計方法,A
2602,D-2020_U43,11,3,今日の内容で、分からなかったこと・できなかったことを書いてください,窓関数の違い,A
2603,D-2020_U43,11,4,質問があれば書いてください,特にありません,A


## Gradeの数値化

In [93]:
grade_mapping = {
    'A': 1,
    'B': 0,
    'C': -1,
    'D': -2,
    'F': -3
}

In [94]:
df['target'] = df['grade'].replace(grade_mapping)
df = df.drop(columns='grade')
df

,userid,course_number,question_number,question_content,answer_content,target
0,D-2020_U51,1,1,今日の内容を自分なりの言葉で説明してみてください,黄色のｍoodleにログインして、そのあと水色のmoodleにリンク、授業開始までにTeam...,-3
1,D-2020_U51,1,2,今日の内容で、分かったこと・できたことを書いてください,授業の進め方が理解できた。,-3
2,D-2020_U51,1,3,今日の内容で、分からなかったこと・できなかったことを書いてください,とくになし,-3
3,D-2020_U51,1,4,質問があれば書いてください,とくになし,-3
4,D-2020_U51,1,5,今日の授業の感想や反省を書いてください,オンライン授業ならではの利点を活用する工夫で、最初は大丈夫かと不安だったが何とかなりそうだと...,-3
...,...,...,...,...,...,...
2600,D-2020_U43,11,1,今日の内容を自分なりの言葉で説明してみてください,デジタルフィルタについて,1
2601,D-2020_U43,11,2,今日の内容で、分かったこと・できたことを書いてください,フィルタの種類と設計方法,1
2602,D-2020_U43,11,3,今日の内容で、分からなかったこと・できなかったことを書いてください,窓関数の違い,1
2603,D-2020_U43,11,4,質問があれば書いてください,特にありません,1


In [95]:
import requests
import pandas as pd

# Google Apps ScriptのWebアプリケーションのURL
api_url = "https://script.google.com/macros/s/AKfycbwcsQDMowFqaoUqcD3laovQQSbu6JmNDoo9qwqVEba2_1xdvGNxg010f4-y17EtvxI-vA/exec"


#for i in range(10):
for i in df.index:
    text = df.loc[i, 'answer_content']
    params = {
        'text': "\"" + text + "\"",
        'source': 'ja',
        'target': 'en'
    }
    r_post = requests.post(api_url, data=params)
    translated_text = r_post.json()['text']  # レスポンスを解析し、翻訳されたテキストを取得
    df.loc[i, 'translated_content'] = translated_text  # 新しい列に翻訳結果を保存


In [96]:
df

,userid,course_number,question_number,question_content,answer_content,target,translated_content
0,D-2020_U51,1,1,今日の内容を自分なりの言葉で説明してみてください,黄色のｍoodleにログインして、そのあと水色のmoodleにリンク、授業開始までにTeam...,-3,"""Log in to the yellow moodle, then link to the..."
1,D-2020_U51,1,2,今日の内容で、分かったこと・できたことを書いてください,授業の進め方が理解できた。,-3,"""I understood how to proceed with the class."""
2,D-2020_U51,1,3,今日の内容で、分からなかったこと・できなかったことを書いてください,とくになし,-3,"""Nothing in particular"""
3,D-2020_U51,1,4,質問があれば書いてください,とくになし,-3,"""Nothing in particular"""
4,D-2020_U51,1,5,今日の授業の感想や反省を書いてください,オンライン授業ならではの利点を活用する工夫で、最初は大丈夫かと不安だったが何とかなりそうだと...,-3,"""I was worried at first that I would be okay w..."
...,...,...,...,...,...,...,...
2600,D-2020_U43,11,1,今日の内容を自分なりの言葉で説明してみてください,デジタルフィルタについて,1,"""About digital filters"""
2601,D-2020_U43,11,2,今日の内容で、分かったこと・できたことを書いてください,フィルタの種類と設計方法,1,"""Filter Types and Design Methods"""
2602,D-2020_U43,11,3,今日の内容で、分からなかったこと・できなかったことを書いてください,窓関数の違い,1,"""Differences in Window Functions"""
2603,D-2020_U43,11,4,質問があれば書いてください,特にありません,1,"""Nothing in particular"""


In [97]:
df
df.to_csv(OUT_PATH + '/' + file[0] + '_Embedded.csv', index=False)